In [28]:
!git clone https://github.com/JingfengSteven/givemesomecredit

fatal: destination path 'givemesomecredit' already exists and is not an empty directory.


In [29]:
import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
from givemesomecredit import models as nam_models
import os.path as osp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn

##Data reading

In [30]:
orgdata=pd.read_csv("givemesomecredit/cs-training.csv")
orgdata=orgdata.dropna(axis=0)
orgdata.head()

,Unnamed: 0,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0


In [31]:
data=orgdata

In [32]:
x=data.iloc[:,2:data.shape[1]]
y=data.iloc[:,1]
x.insert(data.shape[1]-2, 'label', y)
x.head()

,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,label
0,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0,1
1,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0,0
2,0.658180,38,1,0.085113,3042.0,2,1,0,0,0.0,0
3,0.233810,30,0,0.036050,3300.0,5,0,0,0,0.0,0
4,0.907239,49,1,0.024926,63588.0,7,0,1,0,0.0,0


In [33]:
x=np.array(x)
x.shape

(120269, 11)

##Data cleaning

In [34]:
x[:,[0,2]]=x[:,[2,0]]
x[:,[1,8]]=x[:,[8,1]]
x[:,[2,6]]=x[:,[6,2]]
x[:,[3,4]]=x[:,[4,3]]
x[:,[4,9]]=x[:,[9,4]]

In [35]:
x1=x

##First three feature are 3-pairwise, the last two feature are single

In [36]:
x1 = np.delete(x1, np.where
    (x1[:,0] >= 20), axis=0)
x1 = np.delete(x1, np.where
    (x1[:,1] >= 20), axis=0)
x1 = np.delete(x1, np.where
    (x1[:,2] >= 20), axis=0)

In [37]:
x1.shape

(120121, 11)

In [38]:
x1[:,0][x1[:,0]>=4]=4
x1[:,1][x1[:,1]>=4]=4
x1[:,2][x1[:,2]>=4]=4

In [39]:
x1[:,3][(x1[:,3]>=0)&(x1[:,3]<2500)]=5
x1[:,3][(x1[:,3]>=2500)&(x1[:,3]<5000)]=4
x1[:,3][(x1[:,3]>=5000)&(x1[:,3]<7500)]=3
x1[:,3][(x1[:,3]>=7500)&(x1[:,3]<10000)]=2
x1[:,3][(x1[:,3]>=10000)&(x1[:,3]<50000)]=1
x1[:,3][x1[:,3]>=50000]=0

In [40]:
x1[:,4][x1[:,4]>=5]=5

In [41]:
X=x1[:,0:len(x1[0])-1]
Y=x1[:,-1]

##Build the model

In [42]:
tf.compat.v1.reset_default_graph()
#the change of the structure of NAM is in kwargs)
#in this demo, first three feature are strong monotonicity, thereby, they are combined in a DNN.
Number_of_DNN=8
Number_of_Unit=0
Trainable=True
Use_Shallow=False
Model=nam_models.NAM(Number_of_DNN,Number_of_Unit,Trainable,Use_Shallow,feature_dropout = 0.0,dropout = 0.0,kwargs=[3,1,1,1,1,1,1,1])
Model(X)

<tf.Tensor: shape=(120121,), dtype=float32, numpy=
array([0.02611502, 0.05746063, 0.09515809, ..., 0.04235728, 0.06842479,
       0.03196806], dtype=float32)>

In [43]:
Model.summary()

Model: "nam"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_nn (FeatureNN)      multiple                  21        
                                                                 
 feature_nn_1 (FeatureNN)    multiple                  13        
                                                                 
 feature_nn_2 (FeatureNN)    multiple                  13        
                                                                 
 feature_nn_3 (FeatureNN)    multiple                  13        
                                                                 
 feature_nn_4 (FeatureNN)    multiple                  13        
                                                                 
 feature_nn_5 (FeatureNN)    multiple                  13        
                                                                 
 feature_nn_6 (FeatureNN)    multiple                  13      

##Single monotonicity

In [44]:
def descretize_single(monotonic_feature,lower_bound,upper_bound,spli,X):
  individual_output=[]
  bar=np.zeros(len(monotonic_feature))
  for i in range(len(monotonic_feature)):
    individual_output.append(np.linspace(lower_bound[i], upper_bound[i], num=spli[i]))
  return individual_output
X=X
monotonic_feature=[3,4]
lower_bound=[0,0]
upper_bound=[5,5]
spli=[6,6]
individual_output=descretize_single(monotonic_feature,lower_bound,upper_bound,spli,X)

In [45]:
individual_output

[array([0., 1., 2., 3., 4., 5.]), array([0., 1., 2., 3., 4., 5.])]

##3-strong-pair monotonicity

In [46]:
def descretize_3_pair():
  lower_bound=0
  upper_bound=4
  pair=[]
  pair1=[]
  pair2=[]
  pair3=[]
  for i in range(5):
    for j in range(5):
      for k in range(5):
        pair.append([i,j,k])
        pair1.append([i,j+1,k-1])
        pair2.append([i+1,j-1,k])
        pair3.append([i-1,j,k])
  return pair,pair1,pair2,pair3

In [47]:
#f(pair) is smaller than f(pair1),f(pair2),f(pair3)
pair,pair1,pair2,pair3=descretize_3_pair()

## Single monotonicity in that 3-strong-pair monotonicity

In [48]:
def descretize_3_pair_single():
  pair_s=[]
  pair_s1=[]
  for i in range(5):
    for j in range(5):
      for k in range(4):
        pair_s.append([k+1,i,j])
        pair_s1.append([k,i,j])

  for i in range(5):
    for j in range(5):
      for k in range(4):
        pair_s.append([i,k+1,j])
        pair_s1.append([i,k,j])

  for i in range(5):
    for j in range(5):
      for k in range(4):
        pair_s.append([i,j,k+1])
        pair_s1.append([i,j,k])
  return pair_s, pair_s1

In [49]:
#f(pair_s) is smaller than f(pair_s1)
pair_s, pair_s1=descretize_3_pair_single()

##Pre_training

In [50]:
Model.compile(loss=tf.keras.losses.binary_crossentropy,
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  metrics='AUC')

training_callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(patience = 5, factor = 0.1, min_lr = 0.001, verbose = 1),
    tf.keras.callbacks.EarlyStopping(patience = 10, restore_best_weights = True),
]

Model.fit(X, Y, batch_size=64, epochs=10)

Epoch 1/10
1877/1877 [==============================] - 21s 7ms/step - loss: 0.2161 - auc: 0.7681
Epoch 2/10
1877/1877 [==============================] - 6s 3ms/step - loss: 0.1938 - auc: 0.8370
Epoch 3/10
1877/1877 [==============================] - 7s 4ms/step - loss: 0.1904 - auc: 0.8455
Epoch 4/10
1877/1877 [==============================] - 6s 3ms/step - loss: 0.1893 - auc: 0.8487
Epoch 5/10
1877/1877 [==============================] - 7s 4ms/step - loss: 0.1889 - auc: 0.8489
Epoch 6/10
1877/1877 [==============================] - 6s 3ms/step - loss: 0.1887 - auc: 0.8491
Epoch 7/10
1877/1877 [==============================] - 7s 4ms/step - loss: 0.1885 - auc: 0.8496
Epoch 8/10
1877/1877 [==============================] - 6s 3ms/step - loss: 0.1883 - auc: 0.8501
Epoch 9/10
1877/1877 [==============================] - 7s 4ms/step - loss: 0.1883 - auc: 0.8500
Epoch 10/10
1877/1877 [==============================] - 6s 3ms/step - loss: 0.1883 - auc: 0.8500


## Training to satisfy the monotonicity requirement. Once first 3 loss reach to 0, decreas the learning_r

In [51]:
each_epoch=10
alpha_1=1
alpha_2=1
alpha_3=1
learning_r=0.001

for i in range(each_epoch):
  Model.network_learn(X,Y,monotonic_feature,individual_output,alpha_1,pair,pair1,pair2,pair3,alpha_2,pair_s,pair_s1,alpha_3,learning_r)
  print("------------")

loss of single monotonicity tf.Tensor([0.], shape=(1,), dtype=float32)
loss of strong pairwise monotonicity tf.Tensor([0.52692807], shape=(1,), dtype=float32)
loss of single monotonicity in strong pairwise monotonicity 0
overall loss tf.Tensor([0.7156728], shape=(1,), dtype=float32)
------------
loss of single monotonicity tf.Tensor([0.], shape=(1,), dtype=float32)
loss of strong pairwise monotonicity tf.Tensor([0.5169889], shape=(1,), dtype=float32)
loss of single monotonicity in strong pairwise monotonicity 0
overall loss tf.Tensor([0.7053868], shape=(1,), dtype=float32)
------------
loss of single monotonicity tf.Tensor([0.], shape=(1,), dtype=float32)
loss of strong pairwise monotonicity tf.Tensor([0.5071615], shape=(1,), dtype=float32)
loss of single monotonicity in strong pairwise monotonicity tf.Tensor([8.4877014e-05], shape=(1,), dtype=float32)
overall loss tf.Tensor([0.695379], shape=(1,), dtype=float32)
------------
loss of single monotonicity tf.Tensor([0.], shape=(1,), dtyp

##Once training is complete, save the model

In [52]:
Model.save_weights('./Model/My_model_1')